In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [2]:
# Helper libraries
import pandas as pd
import uproot

# Decorrelation library
import importlib
disco = importlib.import_module('Disco')

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [4]:
####### Part 1. Load ROOT Data #######
file_format = '/work/submit/kyoon/RareHiggs/data/cat_phi/cat_phi_VBF/test/test_mc{}_VBFcat_{}Cat.root'
sgnmc = 1010
channel = 'Phi'
signal = uproot.open(file_format.format(sgnmc, channel))
background1 = uproot.open(file_format.format(6, channel))
background2 = uproot.open(file_format.format(7, channel))
background3 = uproot.open(file_format.format(8, channel))
background4 = uproot.open(file_format.format(9, channel))

In [5]:
variables = ['HCandMass',
             'HCandPT',
             'goodMeson_iso',
             'zepVar',
             'w']

In [6]:
# Convert to pandas dataframe
signal_df = signal['events'].arrays(variables, library='pd')
background1_df = background1['events'].arrays(variables, library='pd')
background2_df = background2['events'].arrays(variables, library='pd')
background3_df = background3['events'].arrays(variables, library='pd') 
background4_df = background4['events'].arrays(variables, library='pd')

In [7]:
# Add truth values
signal_df['y_true'] = 1
background1_df['y_true'] = 0
background2_df['y_true'] = 0
background3_df['y_true'] = 0
background4_df['y_true'] = 0

In [8]:
# Combine seperate dataframes into one object
complete_df = pd.concat([signal_df,
                         background1_df,
                         background2_df,
                         background3_df,
                         background4_df])

In [9]:
# Shuffle in random order
complete_df = complete_df.sample(frac=1).reset_index(drop=True)

In [10]:
####### Part 2. Prepare Data #######
split_level = 0.7
train_df = complete_df.sample(frac=split_level, random_state=137)
test_df = complete_df.drop(train_df.index)

In [11]:
labels = ['y_true', 'HCandMass', 'w']
train_labels_df = pd.concat([train_df.pop(label) for label in labels], axis=1)
test_labels_df = pd.concat([test_df.pop(label) for label in labels], axis=1)

In [12]:
train_df.shape, train_labels_df.shape, test_df.shape, test_labels_df.shape

((85492, 3), (85492, 3), (36640, 3), (36640, 3))

In [13]:
# Convert to torch-readable format
train_X = torch.tensor(train_df.values).type(torch.float)
train_Y = torch.tensor(train_labels_df.values).type(torch.float)
test_X = torch.tensor(test_df.values).type(torch.float)
test_Y = torch.tensor(test_labels_df.values).type(torch.float)

In [14]:
i = (train_Y[:,0] == 0).nonzero()
len(train_Y[:,0][i])

697

In [34]:
# Standardize input
BATCH_SIZE = 64

In [35]:
# Define custom dataloaders
class TrainData(Dataset):
    def __init__(self, X_data, Y_data, other_data):
        self.X_data = X_data
        self.Y_data = Y_data
        self.other_data = other_data
    def __getitem__(self, index):
        return self.X_data[index], self.Y_data[index],  self.other_data[index]
    def __len__ (self):
        return len(self.X_data)

class TestData(Dataset):
    def __init__(self, X_data, Y_data):
        self.X_data = X_data
        self.Y_data = Y_data
    def __getitem__(self, index):
        return self.X_data[index], self.Y_data[index]
    def __len__ (self):
        return len(self.X_data)
    
train_data = TrainData(torch.FloatTensor(train_X), 
                       torch.FloatTensor(train_Y[:,0]),
                       torch.FloatTensor(train_Y[:,1:]))
test_data = TestData(torch.FloatTensor(test_X),
                     torch.FloatTensor(test_Y[:,0]))

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [17]:
####### Part N. Build Neural Network #######
class Net(nn.Module):
    def __init__(self, input_shape):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_shape, 9),
            nn.ReLU(),
            nn.Linear(9, 9),
            nn.ReLU(),
            nn.Linear(9, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [18]:
model = Net(3).to(device)
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3, out_features=9, bias=True)
    (1): ReLU()
    (2): Linear(in_features=9, out_features=9, bias=True)
    (3): ReLU()
    (4): Linear(in_features=9, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [19]:
EPOCHS = 5
LEARNING_RATE = 0.01

In [21]:
####### Part N. Build Custom Loss #######
def my_loss(y_pred, y_true, mass_tensor=[], weight_tensor=[], penalty_rate=0.1):
    loss_term = nn.functional.binary_cross_entropy(y_pred, y_true)
    bkg_indices = (y_true==0).nonzero()
    if len(bkg_indices) <= 2:
        return loss_term
    else:
        bkg_masses = mass_tensor[bkg_indices]
        bkg_weights = weight_tensor
        if torch.is_tensor(weight_tensor):
            bkg_weights = weight_tensor[bkg_indices]
        regularization_term = penalty_rate * disco.distance_corr(y_pred, mass_tensor, weight_tensor)
        print(regularization_term, loss_term)
        return loss_term + penalty_rate * regularization_term

In [22]:
####### Part N. Train Neural Network #######
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [43]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for batch_X, batch_Y, batch_other in train_loader:
        batch_X, y_true, batch_other = batch_X.to(device), batch_Y.to(device), batch_other.to(device)
        optimizer.zero_grad()
        
        y_pred = model(batch_X)
        y_pred = torch.full((len(batch_X), 1), .5, requires_grad=True)
        
        loss = my_loss(y_pred, y_true.unsqueeze(1),
                       batch_other[:,0].unsqueeze(1),
                       1, 1)
                       # weight is 1 until I figure this out batch_other[:,1].unsqueeze(1))
        acc = binary_acc(y_pred, y_true.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan, grad_fn=<MulBackward0>) tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(nan

In [54]:
####### Part N. Test Model #######
y_pred_list = []
y_true_list = []
model.eval()
with torch.no_grad():
    for batch_X, batch_Y in test_loader:
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
        y_test_pred = model(batch_X)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag)
        y_true_list.append(batch_Y)
y_check_pred_list = torch.eq(torch.cat(y_pred_list), torch.cat(y_true_list))

In [55]:
y_check_pred_list

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [95]:
m = torch.jit.script(model)
torch.jit.save(m, "modelClassification.pt")
print(m)

RecursiveScriptModule(
  original_name=Net
  (flatten): RecursiveScriptModule(original_name=Flatten)
  (linear_relu_stack): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=ReLU)
    (4): RecursiveScriptModule(original_name=Linear)
    (5): RecursiveScriptModule(original_name=Sigmoid)
  )
)
